# Merge Metadata

- pull from downloaded google sheets of FAIRe terms

### Import

In [1]:
import pandas as pd
import numpy as np
import glob

### Import Metadata files

In [46]:
#look at test file
file = '/Users/kpitz/github/GLOMICON/intercomparison/metadata/MBARI _ NOAA_MIMARKS.survey.water.6.0_sharing.xlsx'
xl = pd.ExcelFile(file)
print('Sheets in excel file:',xl.sheet_names)  # see all sheet names
#sheets = xl.sheet_names
sheets = ['study_data', 'water_sample_data', 'amplicon_prep_data', 'analysis_data']
dfs= []
skiprow_num = [5,8,6,6]
for sheet, sr in zip(sheets, skiprow_num):
    print(sheet,sr)
    df = xl.parse(sheet,skiprows=sr)
    df = df.dropna(how='all', axis=1)
    df = df.dropna(how='all', axis=0)
    df['sheet'] = sheet
    dfs.append(df)
df.head()

Sheets in excel file: ['README', 'study_data', 'water_sample_data', 'amplicon_prep_data', 'metag_prep_data', 'analysis_data', 'SRA_template', 'measurement_metadata', 'SRA Terms', 'Version log']
study_data 5
water_sample_data 8
amplicon_prep_data 6
analysis_data 6


,amplicon_sequenced,trim_method,cluster_method,pid_clustering,taxa_class_method,taxa_ref_db,code_repo,date_modified,sheet
0,18S V9,atropos,dada2,ASV,BLAST; MEGAN LCA,genbank NR,https://github.com/MBARI-BOG/BOG-Banzai-Dada2-...,2024-08-15 08:44:22.928,analysis_data


In [86]:
def water_amplicon_merge(water_df, amplicon_df):
    df = water_df.set_index('sample_name')
    df = df.drop(['sheet', 'date_modified','modified_by'], axis=1)
    # add amplicon metadata to water sample metadata (some water samples don't have amplicon data if not sequenced in institute)
    df = df.join(amplicon_df.set_index('sample_name'), rsuffix='_amplicon_sheet')
    df = df.drop(['sheet', 'date_modified','modified_by'], axis=1)
    return df
def study_data_merge(study_meta, bioinf_meta, institute_name):
    df = pd.concat([study_meta, bioinf_meta], axis=1)
    df = df.drop(['sheet', 'date_modified'], axis=1)
    df = df.T
    df = df.rename(columns={0:institute_name})
    return df
    

In [87]:
df = study_data_merge(dfs[0], dfs[3], 'MBARI')
df.head()

,MBARI
project_id,GLOMICON_INTERCOMP
project_name,Intercomparison of filtered seawater samples a...
project_id_external,GLOMICON_INTERCOMP
project_contact,"Kathleen Pitz, kpitz@mbari.org, https://orcid...."
project_description,"Replicate filters, originating from surface wa..."


In [88]:
df = water_amplicon_merge(dfs[1], dfs[2])
df.head()

,Collecting Institute,Analyzing Institute,cruise_id,station,ctd_bottle_no,sample_replicate,source_material_id,extract_number,sample_title,project_id,...,seq_meth,amplicon_sequenced,target_gene,target_subfragment,pcr_primer_forward,pcr_primer_reverse,pcr_primer_name_forward,pcr_primer_name_reverse,pcr_primer_reference,pcr_cond
sample_name,,,,,,,,,,,,,,,,,,,,,
CN18Sc37_12_Rep_Stdy5_AO,MBARI,MBARI,CN18S,TS,12.0,5.0,CN18Sc37_12_Rep_Stdy5,AO9,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,Illumina MiSeq 2x150,18S V9,18S rRNA,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2
CN18Sc37_12_Rep_Stdy10_AO,MBARI,MBARI,CN18S,TS,12.0,10.0,CN18Sc37_12_Rep_Stdy10,AO10,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,Illumina MiSeq 2x150,18S V9,18S rRNA,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2
CN18Sc37_12_Rep_Stdy14_AO,MBARI,MBARI,CN18S,TS,12.0,14.0,CN18Sc37_12_Rep_Stdy14,AO11,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,Illumina MiSeq 2x150,18S V9,18S rRNA,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2
CN18Sc37_12_Rep_Stdy18_AO,MBARI,MBARI,CN18S,TS,12.0,18.0,CN18Sc37_12_Rep_Stdy18,AO12,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,Illumina MiSeq 2x150,18S V9,18S rRNA,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2
CN18Sc37_12_Rep_Stdy22_AO,MBARI,MBARI,CN18S,TS,12.0,22.0,CN18Sc37_12_Rep_Stdy22,AO13,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,Illumina MiSeq 2x150,18S V9,18S rRNA,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2


In [62]:
df = dfs[1].copy()
df.set_index('sample_name', inplace=True)
# add amplicon metadata to water sample metadata (some water samples don't have amplicon data if not sequenced in institute)
df = df.join(dfs[2].set_index('sample_name'), rsuffix='_amplicon_sheet')
print(list(df))

df.head()

['Collecting Institute', 'Analyzing Institute', 'cruise_id', 'station', 'ctd_bottle_no', 'sample_replicate', 'source_material_id', 'extract_number', 'sample_title', 'project_id', 'organism', 'collection_date_local', 'collection_date', 'depth', 'env_broad_scale', 'env_local_scale', 'env_medium', 'geo_loc_name', 'waterBody', 'countryCode', 'decimalLatitude', 'decimalLongitude', 'samp_vol_we_dna_ext', 'samp_collect_device', 'samp_mat_process', 'size_frac', 'sample_type', 'collection_method', 'basisOfRecord', 'chlorophyll', 'density', 'fluor', 'pressure', 'salinity', 'samp_store_temp', 'temp', 'date_modified', 'modified_by', 'sheet', 'library_id', 'title', 'library_strategy', 'library_source', 'library_selection', 'lib_layout', 'platform', 'instrument_model', 'filetype', 'filename', 'filename2', 'seq_facility', 'seq_meth', 'amplicon_sequenced', 'target_gene', 'target_subfragment', 'pcr_primer_forward', 'pcr_primer_reverse', 'pcr_primer_name_forward', 'pcr_primer_name_reverse', 'pcr_primer_

,Collecting Institute,Analyzing Institute,cruise_id,station,ctd_bottle_no,sample_replicate,source_material_id,extract_number,sample_title,project_id,...,target_subfragment,pcr_primer_forward,pcr_primer_reverse,pcr_primer_name_forward,pcr_primer_name_reverse,pcr_primer_reference,pcr_cond,date_modified_amplicon_sheet,modified_by_amplicon_sheet,sheet_amplicon_sheet
sample_name,,,,,,,,,,,,,,,,,,,,,
CN18Sc37_12_Rep_Stdy5_AO,MBARI,MBARI,CN18S,TS,12.0,5.0,CN18Sc37_12_Rep_Stdy5,AO9,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,2024-08-15 08:42:28.469,NaN,amplicon_prep_data
CN18Sc37_12_Rep_Stdy10_AO,MBARI,MBARI,CN18S,TS,12.0,10.0,CN18Sc37_12_Rep_Stdy10,AO10,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,2024-06-18 19:31:14.358,kpitz@mbari.org,amplicon_prep_data
CN18Sc37_12_Rep_Stdy14_AO,MBARI,MBARI,CN18S,TS,12.0,14.0,CN18Sc37_12_Rep_Stdy14,AO11,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,NaT,NaN,amplicon_prep_data
CN18Sc37_12_Rep_Stdy18_AO,MBARI,MBARI,CN18S,TS,12.0,18.0,CN18Sc37_12_Rep_Stdy18,AO12,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,NaT,NaN,amplicon_prep_data
CN18Sc37_12_Rep_Stdy22_AO,MBARI,MBARI,CN18S,TS,12.0,22.0,CN18Sc37_12_Rep_Stdy22,AO13,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,NaT,NaN,amplicon_prep_data


In [71]:
# study-wide metadata can organize by institution?
df = pd.concat([dfs[3], dfs[0]], axis=1)
print(list(df))
df = df.drop(['sheet', 'date_modified'], axis=1)
df = df.T
df = df.rename(columns={0:'MBARI'})

df

['amplicon_sequenced', 'trim_method', 'cluster_method', 'pid_clustering', 'taxa_class_method', 'taxa_ref_db', 'code_repo', 'date_modified', 'sheet', 'project_id', 'project_name', 'project_id_external', 'project_contact', 'project_description', 'type', 'license', 'associated_parties', 'study_area_description', 'date_modified', 'sheet']


,MBARI
amplicon_sequenced,18S V9
trim_method,atropos
cluster_method,dada2
pid_clustering,ASV
taxa_class_method,BLAST; MEGAN LCA
taxa_ref_db,genbank NR
code_repo,https://github.com/MBARI-BOG/BOG-Banzai-Dada2-...
project_id,GLOMICON_INTERCOMP
project_name,Intercomparison of filtered seawater samples a...
project_id_external,GLOMICON_INTERCOMP


In [65]:
dfs[0]

,project_id,project_name,project_id_external,project_contact,project_description,type,license,associated_parties,study_area_description,date_modified,sheet
0,GLOMICON_INTERCOMP,Intercomparison of filtered seawater samples a...,GLOMICON_INTERCOMP,"Kathleen Pitz, kpitz@mbari.org, https://orcid....","Replicate filters, originating from surface wa...",Occurrence,CC-BY 4.0,"Francisco Chavez, chfr@mbari.org",eDNA from north of Monterey Bay collected duri...,2024-08-22 14:14:36.023,study_data


In [55]:
# Join two sample_name dfs:
#df = pd.concat([dfs[1].set_index('sample_name'), dfs[2].set_index('sample_name')], axis=0)
df = dfs[1].copy()
df['dup'] = df.duplicated('sample_name', keep=False)
df = df.loc[df['source_material_id'].isna()==False]
#df = df.dropna('source_material_id')
df = df.sort_values('dup')

df

,sample_name,Collecting Institute,Analyzing Institute,cruise_id,station,ctd_bottle_no,sample_replicate,source_material_id,extract_number,sample_title,...,density,fluor,pressure,salinity,samp_store_temp,temp,date_modified,modified_by,sheet,dup
0,CN18Sc37_12_Rep_Stdy5_AO,MBARI,MBARI,CN18S,TS,12.0,5.0,CN18Sc37_12_Rep_Stdy5,AO9,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,2024-07-31 12:16:48.569,NaN,water_sample_data,False
37,ROS-MBA12,MBARI,SBR,CN18S,TS,12.0,NaN,CN18Sc37_12_Rep_Stdy12,NaN,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,NaT,NaN,water_sample_data,False
36,Davenport_11_0008 | Davenport_11_0049,MBARI,AWI,CN18S,TS,12.0,NaN,CN18Sc37_12_Rep_Stdy11,NaN,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,NaT,NaN,water_sample_data,False
42,Davenport_19_0008 | Davenport_19_0049,MBARI,AWI,CN18S,TS,12.0,NaN,CN18Sc37_12_Rep_Stdy19,NaN,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,NaT,NaN,water_sample_data,False
43,ROS-MBA20,MBARI,SBR,CN18S,TS,12.0,NaN,CN18Sc37_12_Rep_Stdy20,NaN,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,NaT,NaN,water_sample_data,False
33,ROS-MBA07,MBARI,SBR,CN18S,TS,12.0,NaN,CN18Sc37_12_Rep_Stdy7,NaN,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,NaT,NaN,water_sample_data,False
32,Davenport_06_0008 | Davenport_06_0049,MBARI,AWI,CN18S,TS,12.0,NaN,CN18Sc37_12_Rep_Stdy6,NaN,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,NaT,NaN,water_sample_data,False
39,Davenport_15_0008 | Davenport_15_0049,MBARI,AWI,CN18S,TS,12.0,NaN,CN18Sc37_12_Rep_Stdy15,NaN,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,NaT,NaN,water_sample_data,False
40,ROS-MBA16,MBARI,SBR,CN18S,TS,12.0,NaN,CN18Sc37_12_Rep_Stdy16,NaN,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,NaT,NaN,water_sample_data,False
29,Davenport_02_0008 | Davenport_02_0049,MBARI,AWI,CN18S,TS,12.0,NaN,CN18Sc37_12_Rep_Stdy2,NaN,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,NaT,NaN,water_sample_data,False


In [47]:
dfs[2].head()

,sample_name,library_id,title,library_strategy,library_source,library_selection,lib_layout,platform,instrument_model,filetype,...,target_subfragment,pcr_primer_forward,pcr_primer_reverse,pcr_primer_name_forward,pcr_primer_name_reverse,pcr_primer_reference,pcr_cond,date_modified,modified_by,sheet
0,CN18Sc37_12_Rep_Stdy5_AO,AO9,18S amplicon metabarcoding GLOMICON comparison...,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,2024-08-15 08:42:28.469,NaN,amplicon_prep_data
1,CN18Sc37_12_Rep_Stdy10_AO,AO10,18S amplicon metabarcoding GLOMICON comparison...,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,2024-06-18 19:31:14.358,kpitz@mbari.org,amplicon_prep_data
2,CN18Sc37_12_Rep_Stdy14_AO,AO11,18S amplicon metabarcoding GLOMICON comparison...,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,NaT,NaN,amplicon_prep_data
3,CN18Sc37_12_Rep_Stdy18_AO,AO12,18S amplicon metabarcoding GLOMICON comparison...,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,NaT,NaN,amplicon_prep_data
4,CN18Sc37_12_Rep_Stdy22_AO,AO13,18S amplicon metabarcoding GLOMICON comparison...,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,NaT,NaN,amplicon_prep_data


In [48]:
dfs[1].head()

,sample_name,Collecting Institute,Analyzing Institute,cruise_id,station,ctd_bottle_no,sample_replicate,source_material_id,extract_number,sample_title,...,chlorophyll,density,fluor,pressure,salinity,samp_store_temp,temp,date_modified,modified_by,sheet
0,CN18Sc37_12_Rep_Stdy5_AO,MBARI,MBARI,CN18S,TS,12.0,5.0,CN18Sc37_12_Rep_Stdy5,AO9,Seawater sample Monterey Bay (GLOMICON),...,8.1461,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,2024-07-31 12:16:48.569,NaN,water_sample_data
1,CN18Sc37_12_Rep_Stdy10_AO,MBARI,MBARI,CN18S,TS,12.0,10.0,CN18Sc37_12_Rep_Stdy10,AO10,Seawater sample Monterey Bay (GLOMICON),...,8.1461,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,2024-06-18 19:04:45.094,kpitz@mbari.org,water_sample_data
2,CN18Sc37_12_Rep_Stdy14_AO,MBARI,MBARI,CN18S,TS,12.0,14.0,CN18Sc37_12_Rep_Stdy14,AO11,Seawater sample Monterey Bay (GLOMICON),...,8.1461,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,2024-06-18 19:04:45.094,kpitz@mbari.org,water_sample_data
3,CN18Sc37_12_Rep_Stdy18_AO,MBARI,MBARI,CN18S,TS,12.0,18.0,CN18Sc37_12_Rep_Stdy18,AO12,Seawater sample Monterey Bay (GLOMICON),...,8.1461,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,2024-06-18 19:05:42.928,kpitz@mbari.org,water_sample_data
4,CN18Sc37_12_Rep_Stdy22_AO,MBARI,MBARI,CN18S,TS,12.0,22.0,CN18Sc37_12_Rep_Stdy22,AO13,Seawater sample Monterey Bay (GLOMICON),...,8.1461,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,2024-06-18 19:04:45.094,kpitz@mbari.org,water_sample_data


In [37]:
df = pd.concat(dfs, axis=0)
df = df.dropna(how='all', axis=1)
df = df.dropna(how='all', axis=0)
df = df.T
df

,0,0,1,2,3,4,5,6,7,8,...,19,20,21,22,23,24,25,26,27,0
project_id,GLOMICON_INTERCOMP,GLOMICON,GLOMICON,GLOMICON,GLOMICON,GLOMICON,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
project_name,Intercomparison of filtered seawater samples a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
project_id_external,GLOMICON_INTERCOMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
project_contact,"Kathleen Pitz, kpitz@mbari.org, https://orcid....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
project_description,"Replicate filters, originating from surface wa...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cluster_method,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dada2
pid_clustering,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASV
taxa_class_method,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BLAST; MEGAN LCA
taxa_ref_db,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,genbank NR
